# [高火力 DOK](https://www.sakura.ad.jp/koukaryoku-dok/) ハンズオン 〜LLM編〜

## ステップ1: このファイルをJupyterLabで読み込む

1. 右上↗️にある "Raw" をクリック（このファイルのソースが表示される）
1. 「高火力 DOK」のノートブック機能でJupyterLabを起動
1. "File" → "Open from URL…" を選択
1. URL入力欄に、手順1で表示したソースのURLを貼り付け

このファイルがJupyterLabに読み込まれたら、ステップ1は終了です。続いて下のステップ2に進んでください。

In [ ]:
%pip install transformers accelerate sentencepiece

## ステップ2: プロンプトを調整

下記ソースコードの `prompt` 変数を好きなメッセージに変更してください。

好きなものに変えて構いませんが、チャットのような会話のキャッチボールをしたい場合は `[INST]` / `[/INST]` の部分は消さないでください。
これを消すと、会話ではなく入力された文字列に続く文章が生成されます。
たとえば「日本は」という文字列だけにすると、「日本はなんとかかんとか」みたいな文章が出てきます。

`prompt` を変えたら、ステップ2は終了です。続いて下のステップ3に進んでください。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# モデル名
model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"

# モデル名に対応したトークナイザーとモデルを読み込み（日本語対応）
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else "auto",
)

# ELYZA指定のフォーマットに合わせたプロンプト
prompt = """[INST] 東京スカイツリーについて簡単に教えて。 [/INST]"""

# トークナイズ
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 推論
output_ids = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

# 結果表示
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

## ステップ3: 文書生成！

上部のツールバーから⏩をクリックしてください。
（"Restart Kernel?" のアラートが出たら "Restart" を押してOK）

依存パッケージやチェックポイントなどのダウンロードなどでしばらく時間がかかります。
ダウンロードが終わったらいよいよ画像生成が始まり、質問に対する応答（または続きの文章）が出てきます。

もしかしたらトンチンカンな答えが返ってくるかもしれませんが、それがハルシネーション（幻覚）です。

以上でハンズオンは終了です。お疲れ様でした！

## おまけ

<https://github.com/shimataro/llm-chat> に、もう少し肉付けしたアプリケーションを置いています。

- 一度応答したら終了ではなく、実際のチャットのように繰り返し質問できる
- 文章全体が生成されるまで待たされるのではなく、生成される過程もストリーム表示される
- テキスト生成だけでなく、Seq2Seqモデル（翻訳やText2Text）も使える
- 認証が必要なモデルも使える

モデルへの問い合わせなど基本部分は今回の内容と同じなので、ぜひ中を覗いてみてください。